In [1]:
import ssl
import sys
import psycopg2
from sqlalchemy import create_engine
import paho.mqtt.client
import json
import pandas as pd

In [2]:
def pretty_select(query):
    connection_string = 'postgres://{}:{}@{}:5432/{}'.format(user, password, host, dbname)
    try:
        engine = create_engine(connection_string)
        records = pd.read_sql_query(query, engine)
    except Exception as e:
        print('Error en el query:', e)
    else:
        return records

In [3]:
def select(query):
    cur = myConnection.cursor()
    try:
        cur.execute(query)
    except Exception as e:
        myConnection.commit()
        print('Error en el query:', e)
    else:
        records = cur.fetchall()
        cur.close()
        return records

In [4]:
host='ruby.db.elephantsql.com'
user ='uicdhpnp'
password='Kfp61NZwnYQVCSDf-zl7Jae836R2u0Fn'
dbname='uicdhpnp'

In [61]:
myConnection.close()

In [5]:
myConnection = psycopg2.connect(host = host,
                                user= user, password =password,
                                dbname= dbname)

In [ ]:
query = """create table sensores_data (
	ID_Estante varchar (4),
    cant numeric (3,0),
    temp numeric (3,1),
    hora character varying,
    primary key(ID_Estante, hora)
);"""
cur = myConnection.cursor()
cur.execute(query)
cur.close()
myConnection.commit()

In [ ]:
query = """insert into Compra (ID_Compra, ID_Cliente, fecha, ID_Product, cant, ID_Sucursal) values (6003, 25021709, '11/5/2019', 3001, 3, 1000);
insert into Compra (ID_Compra, ID_Cliente, fecha, ID_Product, cant, ID_Sucursal) values (6003, 25021709, '11/5/2019', 3004, 2, 1000);
insert into Compra (ID_Compra, ID_Cliente, fecha, ID_Product, cant, ID_Sucursal) values (6003, 25021709, '11/5/2019', 3010, 3, 1000);
insert into Compra (ID_Compra, ID_Cliente, fecha, ID_Product, cant, ID_Sucursal) values (6003, 25021709, '11/5/2019', 3006, 1, 1000);
insert into Compra (ID_Compra, ID_Cliente, fecha, ID_Product, cant, ID_Sucursal) values (6003, 25021709, '11/5/2019', 3015, 4, 1000);"""

cur = myConnection.cursor()
cur.execute(query)

In [ ]:
query = """UPDATE compra
 SET id_sucursal='1001' where id_compra like '6002'
 """

In [ ]:
query = """select * from compra"""
pretty_select(query)

In [ ]:
#Cuales son los clientes conocidos y desconocidos que mas han comprado en cada una de las tiendas
AÑADIR DATA EN OTRA TIENDA PARA AGREGAR ESO

In [6]:
query = """select x.id_cliente, cliente.nombre, cliente.apellido,
            case when afiliado.id_cliente is null then 'NO AFILIADO' else 'AFILIADO' end as afiliacion
            from 
            (SELECT compra.id_cliente, COUNT(DISTINCT (compra.id_cliente, id_compra)) as compritas,
            row_number() over 
            (partition by case when afiliado.id_cliente is null then 'NO AFILIADO' else 'AFILIADO' end) as top
            FROM compra left join afiliado on compra.id_cliente = afiliado.id_cliente
            group by compra.id_cliente, afiliado.id_cliente
            order by compritas DESC) x
            left join afiliado on x.id_cliente = afiliado.id_cliente
            inner join cliente on x.id_cliente = cliente.id_cliente
            where top <=1"""
pretty_select(query)

,id_cliente,nombre,apellido,afiliacion
0,25021709,Hestia,Boggish,AFILIADO


In [7]:
#CONOCIDOS
query = """SELECT id_cliente, COUNT(DISTINCT (compra.id_cliente, id_compra)) as compritas
           FROM compra group by id_cliente
           order by compritas DESC"""
pretty_select(query)

,id_cliente,compritas
0,25021709,3
1,25399720,1


In [8]:
#DESCONOCIDOS
query = """SELECT compra.id_cliente, COUNT(DISTINCT (compra.id_cliente, id_compra)) as compritas
        FROM compra full join afiliado on compra.id_cliente=afiliado.id_cliente 
        where compra.id_cliente is null or afiliado.id_cliente is null
        group by compra.id_cliente
        order by compritas desc limit 1"""
pretty_select(query)

,id_cliente,compritas
0,None,1


In [ ]:
#Diga cual es la categoría de producto que menos se ha vendido en cada una de las tiendas.

In [17]:
query = """select distinct on (id_sucursal) id_sucursal, categoria.nombre from compra 
           inner join producto on producto.id_producto = compra.id_producto
           inner join categoria on producto.id_categoria = categoria.id_categoria
           group by categoria.nombre, id_sucursal order by id_sucursal, sum(cant) asc limit 2"""
pretty_select(query)

,id_sucursal,nombre
0,1000,Higiene
1,1001,Snacks


In [ ]:
#Cuales es el top 5 de productos que mas se han vendido en cada una de las tiendas.

In [20]:
query = """select sucursal.direccion, x.nombrecito, x.cantidad from 
           (select id_sucursal, producto.nombre as nombrecito, sum(cant) as cantidad, 
            row_number() over (partition by id_sucursal order by sum(cant) desc) as numero
            from compra 
            inner join producto on compra.id_producto = producto.id_producto
            group by id_sucursal, producto.nombre) x
            inner join sucursal on x.id_sucursal = sucursal.id_sucursal
           where x.numero <= 5 group by sucursal.direccion, x.nombrecito, x.cantidad order by sucursal.direccion, x.cantidad desc"""
pretty_select(query)

,direccion,nombrecito,cantidad
0,las mercedes,Toallitas humedas Huggies,4.0
1,las mercedes,Chistorra La Monserratina,2.0
2,las mercedes,Colgate,2.0
3,las mercedes,Lechuga,2.0
4,las mercedes,Coca cola 1.5,1.0
5,los samanes,Toallitas humedas Huggies,8.0
6,los samanes,Chocolate de leche Nestle,6.0
7,los samanes,Doritos 200,6.0
8,los samanes,Jamon de pavo Movilla,4.0
9,los samanes,Colgate,2.0


In [ ]:
#Diga todos los clientes que en los últimos 7 días a partir de la ejecución del query han comprado
#solo en una tienda y los que han comprado en las dos tiendas (realice queries distintos).

In [11]:
#UNA SOLA TIENDA
query = """select x.id_cliente, cliente.nombre, cliente.apellido from 
            (select id_cliente, id_compra, id_sucursal from compra 
            group by id_compra, fecha, id_cliente, id_sucursal
            having (current_date-compra.fecha)<=400) x
            inner join cliente on x.id_cliente = cliente.id_cliente
            group by x.id_cliente, cliente.nombre, cliente.apellido having count(distinct x.id_sucursal) = 1"""
pretty_select(query)

,id_cliente,nombre,apellido
0,25399720,Edward,Odby


In [21]:
#EN LAS DOS TIENDAS
query = """select x.id_cliente, cliente.nombre, cliente.apellido from 
            (select id_cliente, id_compra, id_sucursal from compra 
            group by id_compra, fecha, id_cliente, id_sucursal
            having (current_date-compra.fecha)<=400) x
            inner join cliente on x.id_cliente = cliente.id_cliente
            group by x.id_cliente, cliente.nombre, cliente.apellido having count(distinct x.id_sucursal) > 1"""
pretty_select(query)

,id_cliente,nombre,apellido
0,25021709,Hestia,Boggish


In [ ]:
#Haga un análisis por estante de cuales categorías de producto tienen mayor rotación de
#inventario y haga una propuesta de modificación

In [ ]:
query = """select """

In [ ]:
#Encuentre a todos los clientes que han pagado con 2 bancos distintos en la ultima semana. 
#Enumérelos y diga si son parte del programa de afiliados.

In [22]:
query = """select row_number() over (order by compra.id_cliente), cliente.nombre, cliente.apellido, 
            case when afiliado.id_cliente is null then 'NO AFILIADO' else 'AFILIADO' end as afiliacion
            from (select id_factura, id_compra, id_banco, fecha from factura 
            group by id_factura, id_compra, id_banco, fecha
            having (current_date-factura.fecha)<=7) x
            full outer join compra on x.id_compra = compra.id_compra
            left join afiliado on compra.id_cliente = afiliado.id_cliente
            inner join cliente on compra.id_cliente = cliente.id_cliente
            group by compra.id_cliente, cliente.nombre, cliente.apellido, afiliado.id_cliente having count(distinct (x.id_banco, compra.id_cliente))>=2"""
pretty_select(query)

,row_number,nombre,apellido,afiliacion


In [ ]:
#Realizar compra: Esto procedimiento almacenado debe tener como parámetro una lista de
#productos con su cantidad, el banco por donde se va a cancelar y un cliente. Debe tener toda la
#lógica para ingresar en las distintas tablas la compra realizada.

In [63]:
query = """create OR REPLACE function realizar_compra (lista VARCHAR[], cantidades INTEGER[], banco VARCHAR, cliente VARCHAR, sucursal VARCHAR) 
       RETURNS NUMERIC as 
    $$
        DECLARE
        
            idcompra VARCHAR := CAST((select MAX(id_compra) from compra) as INTEGER) + 1;
        
            idfactura VARCHAR := CAST((select MAX(id_factura) from factura) as INTEGER) + 1;
            
            precio NUMERIC;
            
            plata NUMERIC := 0;
            
            hoy TIMESTAMP := NOW();
            
            i INTEGER;
        
        BEGIN
                       
            
            FOR i IN array_lower(lista, 1) .. array_upper(lista, 1)
            LOOP
            
                precio:=  (select precio from costo_producto as costo where id_producto=lista[i] AND fecha =(
                select MAX(fecha) from costo_producto as costo2 where costo.id_producto=costo2.id_producto));
                
                plata:= precio*cantidades[i] + plata;
                
                INSERT INTO compra(id_compra, id_cliente, fecha, id_producto, cant, id_factura, id_sucursal) VALUES
                (idcompra, cliente, hoy, lista[i], cantidades[i], idfactura, sucursal);
                
                UPDATE inventario SET cantidad = cantidad - cantidades[i] WHERE id_producto = lista[i];
                
            END LOOP;   
            
            INSERT INTO factura(id_factura, total, fecha, id_banco, id_compra) VALUES (idfactura, plata, hoy, banco, idcompra);
             
            RETURN plata;
        END;
    $$
language plpgsql"""
pretty_select(query)

Error en el query: This result object does not return rows. It has been closed automatically.


In [ ]:
#Haga un procedimiento almacenado que al final del día saque un estado de cuenta de cuanto
#dinero se tiene en cada banco

In [ ]:
#Haga un procedimiento almacenado que al final del mes saque un estado de cuenta de los
#puntos de todos los afiliados al programa de fidelidad que compraron ese mes.

In [ ]:
#Cada vez que entre un cliente conocido, se debe agregar 1 punto a su estado de cuenta del
#programa de fidelidad.

In [ ]:
#Cada vez que un cliente conocido realice una compra, se le agregaran los puntos en su programa
#de fidelidad

In [ ]:
#Si un cliente desconocido con la misma cedula compra mas de 4 veces, automáticamente se
#agregará al programa de fidelidad.